# ANET : With video captioning result: make it as the input for LGI

In [1]:
import json
with open('/saat/anet_train_anno.json', 'r') as f:
    match_with_vidid = json.load(f)

In [10]:
saatid = 0
mapper = {}
for id in list(match_with_vidid.keys()):
    segs = match_with_vidid[id]['timestamps']
    duration = match_with_vidid[id]['duration']
    for seg in segs:
        
        mapper[saatid] = {'id': id, 'timestamps': seg, 'duration': duration}
        saatid += 1

In [7]:
import json
json_pth = '/saat/temp_anet_test.json'
with open(json_pth, 'r') as f:
    saat_res = json.load(f)


In [8]:
preds = saat_res['predictions']

In [15]:
to_write = ''
already_existing = []
anet_json = {}
for pred in preds:
    tmp_write = ''
    imageid = pred['image_id']
    vidid = mapper[imageid]['id']
    caption =pred['caption']
    # print(vidid, caption)
    segment = mapper[imageid]['timestamps']
    duration = mapper[imageid]['duration']
    # tmp_write = vidid + ' ' + str(segment[0]) + ' ' + str(segment[1]) + '##' + caption + '\n'
    # print(tmp_write)
    # to_write += tmp_write

    if vidid in already_existing:
        already_timestamp = anet_json[vidid]['timestamps']
        already_sentences = anet_json[vidid]['sentences']
        already_timestamp.append(segment)
        already_sentences.append(caption)
        anet_json[vidid] = {'duration':duration, 'timestamps': already_timestamp, 'sentences':already_sentences}
        assert len(already_timestamp) == len(already_sentences)
    else :
        anet_json[vidid] = {'duration' : duration, 'timestamps': [segment], 'sentences': [caption]}


    already_existing.append(vidid)
    
    # break

In [16]:
anet_json

{'v_QOlSCBRmfWY': {'duration': 82.73,
  'timestamps': [[0.83, 19.86], [17.37, 60.81], [56.26, 79.42]],
  'sentences': ['a man is doing a horse',
   'a man is jumping',
   'a man is riding a horse']},
 'v_nwznKOuZM7w': {'duration': 31.65,
  'timestamps': [[11.39, 31.65], [0, 15.51]],
  'sentences': ['a dog is looking out of a refrigerator',
   'a dog is looking out of a refrigerator']},
 'v_W19-fYOO9sg': {'duration': 98.15,
  'timestamps': [[65.76, 95.7], [30.92, 72.63]],
  'sentences': ['a man is putting a pan', 'a man is putting a pan']},
 'v_ZJCbT3300OA': {'duration': 124.88,
  'timestamps': [[0, 7.49], [16.23, 51.82], [56.19, 124.88]],
  'sentences': ['a man is riding a car',
   'a dog is running',
   'a man is running a skateboard']},
 'v_bwcgMTHFRV8': {'duration': 136.7,
  'timestamps': [[54.68, 136.7], [0, 46.48], [46.48, 54.68]],
  'sentences': ['a group of people are dancing',
   'a man is riding a motorcycle',
   'a group of people are riding on a motorcycle']},
 'v_Jy9kiITFsj

In [19]:
with open('activitynet_video_cap_gt_bd.json', 'w') as f:
    json.dump(anet_json, f)